In [1]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
clearall()

In [2]:
import warnings
warnings.filterwarnings("ignore")

print('Importing Librairies...')
import warnings
warnings.filterwarnings("ignore")
import ib_insync
from ib_insync import *
import numpy as np
import pandas as pd
import pandas_datareader as web
from colorama import Fore, Back, Style
import time
import datetime as dt
import os
import sys
print('Librairies imported')
print('')

___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor 55\nLumberJack Jyss 5779(c)')
print(Fore.BLUE,'°0Oo_D.A.G._26_oO0°')
print(Fore.YELLOW,Back.BLUE,'PlaySignal IB Version v1.55',Style.RESET_ALL)

print('')
ib = IB()


try:
    LaBase = pd.read_csv('LaBase.csv')
    print('LaBase est chargée :\n',LaBase.tail(2))
    try:
        LaBase = LaBase.drop(['Unnamed: 0'],axis=1)
    except:
        pass
    try:
        LaBase = LaBase.drop(['Date.1'],axis=1)
    except:
        pass
except:
    print("Il n'y a pas de fichier LaBase")
    kill()
stamp = input("Enter the date (YYYY-MM-DD): ")
compte = 'DU1507427'

def conX():
    util.startLoop()  # uncomment this line when in a notebook
    ib.connect('127.0.0.1', 7497, clientId=1)
    if ib.isConnected() == True:
        print('Golem est maintenant connecté à IB')
    return

def disconX():
    ib.disconnect()
    if ib.isConnected() == False:
        print('Golem est maintenant déconnecté de IB')
    return



def relevpoz():
    CPT = []
    QT = []
    TYPE = []
    STOCK = []
    TICK = []
    AVGCOST = []
    globalpoz = ib.positions(account=compte)
    
    if globalpoz == []:
        print("Il n'y a aucune position en cours actuellement.")
        poz = pd.DataFrame({'ALERTE': [101]}) 
    else:
        print('Il y a actuellement ',len(globalpoz),'positions en cours')
        for i in range(0,len(globalpoz)):
            CPT.append(globalpoz[i][0])
            TYPE.append(str(globalpoz[i][1]).split("(")[0])
            if str(globalpoz[i][1]).split("(")[0] == 'Future':
                TICK.append(str(globalpoz[i][1]).split("'")[1])
            elif str(globalpoz[i][1]).split("(")[0] == 'Forex':
                TICK.append(str(globalpoz[i][1]).split("'")[1])
            elif str(globalpoz[i][1]).split("(")[0] == 'Stock':
                TICK.append(str(globalpoz[i][1]).split("'")[1])
            else:
                TICK.append('NA')

            QT.append(globalpoz[i][2])
            AVGCOST.append(globalpoz[i][3])
            poz = pd.DataFrame()
            
            poz['Compte'] = CPT
            poz['Type'] = TYPE
            poz['Ticker'] = TICK
            poz['Quantité'] = QT
            poz['Average Cost'] = AVGCOST
            poz.to_csv('RelevPoz_'+time.strftime("%d_%m_%Y_%H_%M_%S", time.localtime())+'.csv')
    return(poz)

def price_close(LaBase,ticker):
    
    #today = str(dt.date.today())[:5]+str(dt.date.today())[5:7]+str(dt.date.today())[7:10]
    #yesterday = str(dt.date.today()-dt.timedelta(1))[:5]+str(dt.date.today()-dt.timedelta(1))[5:7]+str(dt.date.today()-dt.timedelta(1))[7:10]
    
    #if str(web.DataReader(ticker,'yahoo',yesterday,today).index[-1])[:10] == today:
    #    price = web.DataReader(ticker,'yahoo',yesterday,today).iloc[-2,3]

    #elif str(web.DataReader(ticker,'yahoo',yesterday,today).index[-1])[:10] == yesterday:
    #    price = web.DataReader(ticker,'yahoo',yesterday,today).iloc[-1,3]
    #else:
    #    price = web.DataReader(ticker,'yahoo',yesterday,today).iloc[-1,3]
    
    price = round(LaBase[LaBase['Symbol']==ticker].iloc[-1]['Close'],2)
    
    return(price)

def BUY(contract,quantity,pot):
    #Contract(conId=270639)
    ib.qualifyContracts(contract)
    order = LimitOrder('BUY', quantity, pot,account=compte,outsideRth=True)
    trade = ib.placeOrder(contract, order)
    print(trade)
    return()

def SELL(contract,quantity,pot):
    #Contract(conId=270639)
    ib.qualifyContracts(contract)
    order = LimitOrder('SELL', quantity, pot,account=compte,outsideRth=True)
    trade = ib.placeOrder(contract, order)
    print(trade)
    return()

try:
    disconX()
except:
    pass

try:
    conX()
except:
    pass

today = str(dt.date.today())[:5]+str(dt.date.today())[5:7]+str(dt.date.today())[7:10]
yesterday = str(dt.date.today()-dt.timedelta(1))[:5]+str(dt.date.today()-dt.timedelta(1))[5:7]+str(dt.date.today()-dt.timedelta(1))[7:10]

try:
    
    signals = pd.read_csv('SIGNALS_'+stamp+'.csv')
    try:
        signals = signals.drop('Unnamed: 0',axis=1)
    except:
        pass
    print('Le fichier SIGNALS_'+stamp+'.csv a bien été chargé')
    
    
except:
    sys.exit("\n Golem n'a pas trouvé le fichier SIGNALS_"+stamp+".csv => Arrêt du programme")

poz = relevpoz()


foo = 0
ACTION = []
ASKED_PRICE = []
WHEN = []
OL = []
CL = []
OS = []
CS = []

try:
    signals['Tradé']
except:
    signals['Tradé'] = 'NA'
if (signals[signals['Tradé']=='NA'].shape[0] == signals.shape[0]):
    print('Passage')
else:
    
    foo = signals[signals['Tradé']=='NA'].index[0]
    print('On reprend le passage en ligne ',foo)
    
try:
    pactol = float(str(str([v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']).split()[2]).split("'")[1])
except:
    print('Montant du pactol non récupéré')
    ib.isConnected()
print(Back.BLUE,Fore.YELLOW,'Les fonds disponibles sur le compte sont de : $',pactol,Style.RESET_ALL)
mise = pactol * 0.02
tmps55=time.time()
for i in range (foo,signals.shape[0]):
    #try:
    #Contract(conId=270639)
    horo = signals.iloc[i,0]
    ticker = signals.iloc[i,1]
    openlong = signals.iloc[i,2]
    closelong = signals.iloc[i,3]
    openshort = signals.iloc[i,4]
    closeshort = signals.iloc[i,5]
    contract = Stock(ticker,'SMART','USD')
    price = price_close(LaBase,ticker)

    en_poz = 0
    if poz.iloc[0,0] != 101:
        if len(poz[ticker == poz.iloc[:,2]]) > 1:
            exit("Il y a plus d'une position pour le ticker ",ticker)
        elif len(poz[ticker == poz.iloc[:,2]]) == 1:
            indeX = int(str(str(poz[ticker == poz.iloc[:,2]].index).split('[')[1]).split(']')[0])
            quantity = abs(poz.iloc[indeX,3])
            if poz.iloc[indeX,3] < 0:
                en_poz = -1
            if poz.iloc[indeX,3] > 0:
                en_poz = 1

    if openlong == 1 and en_poz == 0:
        print('')
        pot = round(price * 1.002,2)
        quantity = round(mise/pot)
        print(Fore.GREEN,'Open Long',Style.RESET_ALL,' sur le ticker ',ticker,' basé sur un close limité à ', price,'\n')
        WHEN.append(time.strftime("%d-%m-%Y %H:%M:%S", time.localtime()))
        ASKED_PRICE.append(pot)
        ACTION.append('Open Long')
        OL.append(1)
        BUY(contract,quantity,pot)


    elif openshort == 1 and en_poz == 0:
        print('')
        pot = round(price * 0.998,2)
        quantity = round(mise/pot)
        print(Fore.GREEN,'Open Short',Style.RESET_ALL,' sur le ticker ',ticker,' basé sur un close limité à ', price,'\n')
        WHEN.append(time.strftime("%d-%m-%Y %H:%M:%S", time.localtime()))
        ASKED_PRICE.append(pot)
        ACTION.append('Open Short')
        OS.append(1)
        SELL(contract,quantity,pot)


    elif closelong == 1 and en_poz == 1:
        print('')
        pot = float(round(poz[poz['Ticker']==ticker]['Average Cost'],2))
        print(Fore.RED,'Close Long',Style.RESET_ALL,' sur le ticker ',ticker,' basé sur un close limité à ', price,'\n')
        WHEN.append(time.strftime("%d-%m-%Y %H:%M:%S", time.localtime()))
        ASKED_PRICE.append(pot)
        ACTION.append('Close Long')
        CL.append(1)
        SELL(contract,quantity,pot)


    elif closeshort == 1 and en_poz == -1:
        print('')
        pot = pot = float(round(poz[poz['Ticker']==ticker]['Average Cost'],2))
        print(Fore.RED,'Close Short',Style.RESET_ALL,' sur le ticker ',ticker,' basé sur un close limité à ', price,'\n')
        WHEN.append(time.strftime("%d-%m-%Y %H:%M:%S", time.localtime()))
        ASKED_PRICE.append(pot)
        ACTION.append('Close Short')
        CS.append(1)
        BUY(contract,quantity,pot)


    else:
        WHEN.append('NA')
        ASKED_PRICE.append('NA')
        ACTION.append('NA')
        
    signals.iloc[foo,7]='Traité'
    #except:
     #   print('souci boucle ',i,' pour le ticker ',ticker)
      #  continue

traded = signals
traded['Action'] = ACTION
traded['Quand'] = WHEN
traded['Prix limite'] = ASKED_PRICE

traded.to_csv('27_OrdresPasses_'+time.strftime("%d_%m_%Y_%H_%M_%S", time.localtime())+'.csv')

tmps22=round(time.time()-tmps55,2)
print("\n Ensemble des opérations du jour effectué en %f" %tmps22,'seconds \n')

print('Il y a eu ',sum(OL),' Open Long, ',sum(OS),' Open Short, ',sum(CL),' Close Long, et ',sum(CS),' Close Short.')
disconX()

Importing Librairies...
Librairies imported

Global Optimized LumberJack Environment Motor 55
LumberJack Jyss 5779(c)
 °0Oo_D.A.G._26_oO0°
  PlaySignal IB Version v1.55 

LaBase est chargée :
               Date Symbol        High         Low        Open       Close  \
408941  2019-12-11    ZTS  123.709999  122.809998  123.199997  123.559998   
408942  2019-12-12    ZTS  125.000000  123.389999  123.690002  124.330002   

           Volume   Adj Close  
408941  1803000.0  123.559998  
408942  2041800.0  124.330002  


Enter the date (YYYY-MM-DD):  2019-12-12


Golem est maintenant déconnecté de IB
Golem est maintenant connecté à IB
Le fichier SIGNALS_2019-12-12.csv a bien été chargé
Il y a actuellement  249 positions en cours
Passage
  Les fonds disponibles sur le compte sont de : $ 494963.9377 

 Close Short   sur le ticker  ADP  basé sur un close limité à  167.16 

Trade(contract=Stock(conId=4661, symbol='ADP', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='ADP', tradingClass='NMS'), order=LimitOrder(orderId=20037, clientId=1, action='BUY', totalQuantity=62.0, lmtPrice=162.82, outsideRth=True, account='DU1507427', conditions=[], softDollarTier=SoftDollarTier()), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 12, 13, 12, 15, 0, 337394, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

 Close Long   sur le ticker  AEP  basé sur un close limité à  91.14 

Trade(contract=Stock(conId=4211, symbol='AEP', exchange='SMART', primaryExchange='NY

Error 201, reqId 20041: Ordre rejeté - Raison :Votre ordre a été rejeté parce que votre actif avec valeur d'emprunt pour [432335.18 USD] ne suffit pas pour couvrir l'exigence de marge initiale de [447994.75 USD].

Canceled order: Trade(contract=Stock(conId=364845429, symbol='AMCR', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='AMCR', tradingClass='AMCR'), order=LimitOrder(orderId=20041, clientId=1, permId=495755483, action='BUY', totalQuantity=925.0, lmtPrice=10.7, auxPrice=0.0, tif='DAY', ocaType=3, outsideRth=True, trailStopPrice=11.7, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='DU1507427', clearingIntent='IB', orderComboLegs=[], adjustedOrderType='None', conditions=[], softDollarTier=SoftDollarTier(), cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(status='Cancelled', remaining=925.0, permId=495755483, clientId=1), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 12, 13, 12, 15, 1, 725913, tzin

Trade(contract=Stock(conId=364845429, symbol='AMCR', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='AMCR', tradingClass='AMCR'), order=LimitOrder(orderId=20041, clientId=1, action='BUY', totalQuantity=925.0, lmtPrice=10.7, outsideRth=True, account='DU1507427', conditions=[], softDollarTier=SoftDollarTier()), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 12, 13, 12, 15, 1, 725913, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

 Open Short   sur le ticker  AMG  basé sur un close limité à  86.94 



...

Trade(contract=Stock(conId=92306678, symbol='RL', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='RL', tradingClass='RL'), order=LimitOrder(orderId=20093, clientId=1, action='SELL', totalQuantity=86.0, lmtPrice=115.52, outsideRth=True, account='DU1507427', conditions=[], softDollarTier=SoftDollarTier()), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 12, 13, 12, 15, 18, 606181, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

 Close Long   sur le ticker  ROK  basé sur un close limité à  205.47 

Trade(contract=Stock(conId=11697, symbol='ROK', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='ROK', tradingClass='ROK'), order=LimitOrder(orderId=20095, clientId=1, action='SELL', totalQuantity=50.0, lmtPrice=198.09, outsideRth=True, account='DU1507427', conditions=[], softDollarTier=SoftDollarTier()), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[Tr

Error 201, reqId 20099: Ordre rejeté - Raison :Votre ordre a été rejeté parce que votre actif avec valeur d'emprunt pour [432148.72 USD] ne suffit pas pour couvrir l'exigence de marge initiale de [442570.25 USD].

Canceled order: Trade(contract=Stock(conId=125980227, symbol='TMUS', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TMUS', tradingClass='NMS'), order=LimitOrder(orderId=20099, clientId=1, permId=495755511, action='SELL', totalQuantity=132.0, lmtPrice=74.74, auxPrice=0.0, tif='DAY', ocaType=3, outsideRth=True, trailStopPrice=75.74, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='DU1507427', clearingIntent='IB', orderComboLegs=[], adjustedOrderType='None', conditions=[], softDollarTier=SoftDollarTier(), cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(status='Cancelled', remaining=132.0, permId=495755511, clientId=1), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 12, 13, 12, 15, 20, 416228,


 Close Long   sur le ticker  WAB  basé sur un close limité à  75.14 

Trade(contract=Stock(conId=661633, symbol='WAB', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='WAB', tradingClass='WAB'), order=LimitOrder(orderId=20101, clientId=1, action='SELL', totalQuantity=125.0, lmtPrice=78.74, outsideRth=True, account='DU1507427', conditions=[], softDollarTier=SoftDollarTier()), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 12, 13, 12, 15, 21, 396592, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

 Close Short   sur le ticker  WBA  basé sur un close limité à  58.58 

Trade(contract=Stock(conId=178634687, symbol='WBA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='WBA', tradingClass='NMS'), order=LimitOrder(orderId=20103, clientId=1, action='BUY', totalQuantity=175.0, lmtPrice=57.26, outsideRth=True, account='DU1507427', conditions=[], softDollarTier=SoftDoll

Error 201, reqId 20105: Ordre rejeté - Raison :Votre ordre a été rejeté parce que votre actif avec valeur d'emprunt pour [432151.35 USD] ne suffit pas pour couvrir l'exigence de marge initiale de [442579.28 USD].

Canceled order: Trade(contract=Stock(conId=13681, symbol='WDC', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='WDC', tradingClass='NMS'), order=LimitOrder(orderId=20105, clientId=1, permId=495755514, action='SELL', totalQuantity=185.0, lmtPrice=53.62, auxPrice=0.0, tif='DAY', ocaType=3, outsideRth=True, trailStopPrice=54.62, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='DU1507427', clearingIntent='IB', orderComboLegs=[], adjustedOrderType='None', conditions=[], softDollarTier=SoftDollarTier(), cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(status='Cancelled', remaining=185.0, permId=495755514, clientId=1), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 12, 13, 12, 15, 22, 376449, tzinf

Trade(contract=Stock(conId=13681, symbol='WDC', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='WDC', tradingClass='NMS'), order=LimitOrder(orderId=20105, clientId=1, action='SELL', totalQuantity=185.0, lmtPrice=53.62, outsideRth=True, account='DU1507427', conditions=[], softDollarTier=SoftDollarTier()), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 12, 13, 12, 15, 22, 376449, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

 Close Long   sur le ticker  XEC  basé sur un close limité à  49.04 

Trade(contract=Stock(conId=16191746, symbol='XEC', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='XEC', tradingClass='XEC'), order=LimitOrder(orderId=20107, clientId=1, action='SELL', totalQuantity=197.0, lmtPrice=50.82, outsideRth=True, account='DU1507427', conditions=[], softDollarTier=SoftDollarTier()), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log

In [ ]:
conX()

In [ ]:
ib.trades()

In [ ]:
disconX()

In [ ]:
conX()

In [ ]:
# MANOUAL

#ticker = 'TROW'
#contract = Stock(ticker,'SMART','USD')
#indeX = int(str(str(poz[ticker == poz.iloc[:,2]].index).split('[')[1]).split(']')[0])
#quantity = abs(poz.iloc[indeX,3])
#price = 117.34
#indeX, quantity, price

In [ ]:
#BUY(contract,quantity,price)
#SELL(contract,quantity,price)

In [ ]:
len(ib.positions(compte))

In [ ]:
ib.positions(compte)

In [ ]:
relevpoz()

In [ ]:
signals.iloc[foo,7]='Traité'

In [ ]:
signals.head()

In [ ]:
util.df(ib.accountSummary(account=compte))

In [ ]:
ib_insync.objects.PnLSingle(compte).dailyPnL

In [ ]:
ib_insync.objects.PnLSingle(compte).unrealizedPnL

In [ ]:
ib_insync.objects.PnLSingle(compte).realizedPnL

In [ ]:
ib_insync.objects.PnLSingle(compte).value

In [ ]:
ib_insync.objects.PnL(compte,'dailyPnL')

In [ ]:
ib_insync.client.Client()#.reqCompletedOrders()

In [ ]:

ib.qualifyContracts(contract)
order = LimitOrder('SELL', quantity, pot,account=compte,conditionsIgnoreRth=True)
trade = ib.placeOrder(contract, order)
print(trade)

In [ ]:
disconX()